# Feature Engineering for Route-Level Analytics

This notebook creates the final, production-ready route-level dataset used for Power BI analysis.

The output of this notebook is a single fact table with one row per route, enriched with decision-oriented features such as efficiency flags, risk buckets, and normalized metrics.


In [5]:
import pandas as pd
import numpy as np

# Load raw data
df=pd.read_csv(r'D:\Coding\last-mile-route-deviation-analytics\data\raw\routes_performance.csv')


In [6]:
# SLA violation at stop level
df["sla_violation"] = df["Arrived Time"] > df["Latest Time"]

# Sequence deviation at stop level
df["sequence_deviation"] = df["IndexP"] != df["IndexA"]


## Feature Engineering Scope

- Grain: One row represents one delivery route.
- This notebook transforms raw stop-level data into route-level metrics.
- All features created here are designed for operational decision-making and dashboarding, not predictive modeling.


In [7]:
route_metrics = (
    df
    .groupby("Route ID")
    .agg(
        total_stops=("Stop ID", "count"),
        planned_distance=("DistanceP", "sum"),
        actual_distance=("DistanceA", "sum"),
        deviation_count=("sequence_deviation", "sum"),
        sla_violation_count=("sla_violation", "sum"),
        route_start_time=("Arrived Time", "min"),
        route_end_time=("Arrived Time", "max")
    )
    .reset_index()
)


In [8]:
# Distance deviation
route_metrics["distance_deviation"] = (
    route_metrics["actual_distance"] -
    route_metrics["planned_distance"]
)

# Efficiency ratio (guard against zero planned distance)
route_metrics["distance_efficiency_ratio"] = (
    route_metrics["actual_distance"] /
    route_metrics["planned_distance"].replace(0, np.nan)
)

# Route duration
route_metrics["route_duration"] = (
    route_metrics["route_end_time"] -
    route_metrics["route_start_time"]
)

# Rates
route_metrics["sla_violation_rate"] = (
    route_metrics["sla_violation_count"] /
    route_metrics["total_stops"].replace(0, 1)
)

route_metrics["deviation_rate"] = (
    route_metrics["deviation_count"] /
    route_metrics["total_stops"].replace(0, 1)
)


In [9]:
route_dim_map = (
    df[["Route ID", "Driver ID", "Week ID", "Country"]]
    .drop_duplicates()
)

route_metrics = route_metrics.merge(
    route_dim_map,
    on="Route ID",
    how="left"
)


## Threshold Selection Logic

The thresholds used in this notebook are business heuristics chosen for interpretability:

- Distance efficiency ratio > 1.1 indicates routes that travel more than 10% extra distance.
- Deviation rate > 0.5 indicates that most stops were reordered.
- SLA violation rate bands (none, low, high) are defined to support executive filtering rather than statistical optimization.

These thresholds are not optimized values and can be adjusted based on business tolerance.


In [10]:
route_metrics["is_inefficient_route"] = (
    route_metrics["distance_efficiency_ratio"] > 1.1
).astype(int)

route_metrics["has_sla_violation"] = (
    route_metrics["sla_violation_count"] > 0
).astype(int)

route_metrics["high_deviation_route"] = (
    route_metrics["deviation_rate"] > 0.5
).astype(int)

route_metrics["positive_deviation_route"] = (
    (route_metrics["distance_deviation"] < 0) &
    (route_metrics["deviation_rate"] > 0)
).astype(int)


## Bucketing Rationale

Continuous metrics are converted into categorical buckets to improve usability in Power BI.

Executives and stakeholders interpret categories more easily than raw ratios, making filters and KPIs clearer and more actionable.


In [ ]:
def efficiency_bucket(ratio):
    if pd.isna(ratio):
        return "unknown"
    elif ratio <= 1.0:
        return "efficient"
    elif ratio <= 1.2:
        return "slightly inefficient"
    else:
        return "highly inefficient"

route_metrics["distance_efficiency_bucket"] = (route_metrics["distance_efficiency_ratio"].apply(efficiency_bucket))


def sla_risk_bucket(rate):
    if rate == 0:
        return "none"
    elif rate <= 0.2:
        return "low"
    else:
        return "high"

route_metrics["sla_risk_level"] = (route_metrics["sla_violation_rate"].apply(sla_risk_bucket))


In [12]:
route_metrics["distance_loss_per_stop"] = (
    route_metrics["distance_deviation"] /
    route_metrics["total_stops"].replace(0, 1)
)


In [13]:
route_metrics = route_metrics.rename(columns={
    "Route ID": "route_id",
    "Driver ID": "driver_id",
    "Week ID": "week_id",
    "Country": "country"
})


In [15]:
route_metrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20218 entries, 0 to 20217
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   route_id                    20218 non-null  int64  
 1   total_stops                 20218 non-null  int64  
 2   planned_distance            20218 non-null  float64
 3   actual_distance             20218 non-null  float64
 4   deviation_count             20218 non-null  int64  
 5   sla_violation_count         20218 non-null  int64  
 6   route_start_time            20218 non-null  float64
 7   route_end_time              20218 non-null  float64
 8   distance_deviation          20218 non-null  float64
 9   distance_efficiency_ratio   20217 non-null  float64
 10  route_duration              20218 non-null  float64
 11  sla_violation_rate          20218 non-null  float64
 12  deviation_rate              20218 non-null  float64
 13  driver_id                   202

In [16]:
route_metrics.head()

,route_id,total_stops,planned_distance,actual_distance,deviation_count,sla_violation_count,route_start_time,route_end_time,distance_deviation,distance_efficiency_ratio,...,driver_id,week_id,country,is_inefficient_route,has_sla_violation,high_deviation_route,positive_deviation_route,distance_efficiency_bucket,sla_risk_level,distance_loss_per_stop
0,0,7,49.468094,44.965197,5,0,42.275,373.553,-4.502897,0.908974,...,0,0,1,0,0,1,1,efficient,none,-0.643271
1,1,7,33.274342,33.610418,6,0,64.855,371.387,0.336076,1.010100,...,1,0,1,0,0,1,0,slightly inefficient,none,0.048011
2,2,7,12.124804,12.508786,4,0,110.283,473.273,0.383982,1.031669,...,2,0,1,0,0,1,0,slightly inefficient,none,0.054855
3,3,10,19.039848,19.374644,6,0,194.274,448.366,0.334795,1.017584,...,3,0,1,0,0,1,0,slightly inefficient,none,0.033480
4,4,8,20.632674,19.528799,2,0,196.588,357.340,-1.103875,0.946499,...,4,0,1,0,0,0,1,efficient,none,-0.137984


## Known Limitations

- Routes with zero planned distance result in undefined efficiency ratios.
- Average metrics may hide route volume differences between drivers.
- SLA violations are influenced by factors beyond distance, such as time windows and stop constraints.

These limitations are acknowledged and will be considered during dashboard interpretation.


In [ ]:
import os

os.makedirs("D:/Coding/last-mile-route-deviation-analytics/data/processed",exist_ok=True)

In [ ]:
route_metrics.to_csv("D:/Coding/last-mile-route-deviation-analytics/data/processed/fact_routes.csv",index=False)

## Output Dataset

The final output is:
- data/processed/fact_routes.csv

This dataset is ready for ingestion into Power BI and serves as the central fact table in the star schema.


In [24]:
# Create dimension tables for Power BI

fact = pd.read_csv("D:/Coding/last-mile-route-deviation-analytics/data/processed/fact_routes.csv")

dim_driver = fact[["driver_id"]].drop_duplicates()
dim_week = fact[["week_id"]].drop_duplicates()
dim_country = fact[["country"]].drop_duplicates()

dim_driver.to_csv("D:/Coding/last-mile-route-deviation-analytics/data/processed/dim_driver.csv", index=False)
dim_week.to_csv("D:/Coding/last-mile-route-deviation-analytics/data/processed/dim_week.csv", index=False)
dim_country.to_csv("D:/Coding/last-mile-route-deviation-analytics/data/processed/dim_country.csv", index=False)